In [1]:
import networkx as nx
from burn_tree import *

In [ ]:
def burn_most_leaves(tree):
    activators = []
    marked = set()
    
    i = 0
    while len(marked) < tree.order():
        # Consider all vertices v of height at most i - which N_i[v] covers the most leaves?
        leaves = get_leaves(tree)
        node_distances = shortest_path_lengths(tree)
        
        # Get all vertices within distance i of a leaf
        near_leaves = set()
        for leaf in leaves:
            nhood = get_neighbourhood(tree, source=leaf, radius=i, node_distances=node_distances)
            for node in nhood:
                near_leaves.add(node)
        
        # For all v we just got: take one with max leaves in its neighbourhood
        max_leaves = 0
        max_node
        for node in near_leaves:
            nhood = get_neighbourhood(tree, source=node, radius=i, node_distances=node_distances)
            
            num_leaves = 0
            for v in nhood:
                if v in leaves:
                    num_leaves += 1
            
            if num_leaves > max_leaves:
                max_node = node
                max_leaves = num_leaves
            
        # Burn that vertex
        activators.insert(0, max_node)
        
        # Remove all vertices we can without disconnecting the graph
        
        
    
    
        i += 1

